In [ ]:
import json
import dask
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from dask.diagnostics import ProgressBar, ResourceProfiler
from multiprocessing.pool import Pool
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import silhouette_samples, silhouette_score

%matplotlib inline

In [ ]:
with open('/Volumes/thesis-data/dtype_dict.json', 'r') as dtypes:
    dtypes = json.load(dtypes)

main = pd.read_csv(
    '/Volumes/thesis-data/cwmtaf.csv',
    dtype=dtypes,
    parse_dates=['ADMDATE', 'DISCDATE', 'PERIOD']
)

In [ ]:
main = main.sample(frac=.1, replace=False)

In [ ]:
groupby = main.groupby('SPELL_ID')['ADM_MET'].nunique()
spells = groupby[groupby.values != 1].index

In [ ]:
multiple_adm_met = main[main['SPELL_ID'].isin(spells)]

multiple_adm_met.set_index(['PATIENT_ID', 'SPELL_ID', 'EPISODE_ID'])\
                .sort_values(['SPELL_ID', 'EPISODE_ID'])[
                    [
                        'PERIOD', 'ADMDATE', 'DISCDATE', 'ADM_MET'
                    ]
                ]

In [ ]:
main = main[-main['SPELL_ID'].isin(spells)]

In [ ]:
main = main[main['ADM_MET'] != '99']

In [ ]:
elective_codes = ['11', '12', '13']
emergency_codes = ['21', '22', '23', '24', '28']
non_elective_codes = emergency_codes + ['31', '32', '81', '82', '83']

In [ ]:
main['Elective'] = main['ADM_MET'].isin(elective_codes)
main['Emergency'] = main['ADM_MET'].isin(emergency_codes)
main['Non-elective'] = main['ADM_MET'].isin(non_elective_codes)

In [ ]:
elective_daycases = main[(main['Elective']) & (main['DC'] == '1')]

elective_daycases_peryear = elective_daycases.groupby('PATIENT_ID')['SPELL_ID'].nunique() \
                            / elective_daycases.groupby('PATIENT_ID')['BENCH_PERIOD'].nunique()

elective_ordinary = main[(main['Elective']) & (main['DC'] == '0')]

elective_ordinary_peryear = elective_ordinary.groupby('PATIENT_ID')['SPELL_ID'].nunique() \
                            / elective_ordinary.groupby('PATIENT_ID')['BENCH_PERIOD'].nunique()

non_elective_daycases = main[(main['Non-elective']) & (main['DC'] == '1')]

non_elective_daycases_peryear = non_elective_daycases.groupby('PATIENT_ID')['SPELL_ID'].nunique() \
                                / non_elective_daycases.groupby('PATIENT_ID')['BENCH_PERIOD'].nunique()

non_elective_ordinary = main[(main['Non-elective']) & (main['DC'] == '0')]

non_elective_ordinary_peryear = non_elective_ordinary.groupby('PATIENT_ID')['SPELL_ID'].nunique() \
                                / non_elective_ordinary.groupby('PATIENT_ID')['BENCH_PERIOD'].nunique()

In [ ]:
names=[
    'Elective daycases per year',
    'Elective ordinary admissions per year',
    'Non-elective daycases per year',
    'Non-elective ordinary admissions per year'
]

In [ ]:
utilisation = pd.concat(
    [
        elective_daycases_peryear,
        elective_ordinary_peryear,
        non_elective_daycases_peryear,
        non_elective_ordinary_peryear
    ],
    axis=1,
    sort=True
)

In [ ]:
utilisation = utilisation.rename({i: name for i, name in enumerate(names)}, axis=1).fillna(0)

In [ ]:
tasks = []
for k in tqdm.tqdm(range(2, 16)):
    km = KMeans(n_clusters=k, n_jobs=4, random_state=0, n_init=50)

    km.fit(utilisation)
    centroids, labels = km.cluster_centers_, km.labels_

    tasks.append(
        [
            dask.delayed(silhouette_score)(utilisation, labels, sample_size=1000),
            dask.delayed(silhouette_samples)(utilisation, labels),
            labels
        ]
    )

In [ ]:
with ProgressBar(), ResourceProfiler() as rprof:
    with dask.config.set(pool=Pool(2), scheduler='processes'):
        results = dask.compute(*tasks)

In [ ]:
for i, n_clusters in enumerate(range(2, 16)):

    fig, ax = plt.subplots(1, figsize=(12, 8), dpi=300)

    ax.set_xlim([-0.1, 1])
    ax.set_ylim([0, len(utilisation) + (n_clusters + 1) * 10])

    silhouette_avg, silhouette_values, cluster_labels = results[i]

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax.set_title("The silhouette plot for the various clusters.")
    ax.set_xlabel("The silhouette coefficient values")
    ax.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax.set_yticks([])  # Clear the yaxis labels / ticks
    ax.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])